In [ ]:
!git clone https://github.com/EmanueleCeglia/PDF-Metadata-Extractor.git

Cloning into 'PDF-Metadata-Extractor'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 37 (delta 10), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (37/37), 465.14 KiB | 19.38 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
%cd PDF-Metadata-Extractor

/content/PDF-Metadata-Extractor


In [ ]:
!pip install -r requirements.txt

In [ ]:
!apt-get install ghostscript -y

## Extract Dates

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from pdf_dates_finder import PDFDatesFinderSemanticSearch  #old method
from pdf_dates_finder import PDFDatesFinderSpace   #new method
import os
import re
from tqdm import tqdm

Download model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B")
model = AutoModelForCausalLM.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B", device_map="auto", load_in_8bit=True)

In [ ]:
def post_process_response_dates(text):
    start_date_pattern = r"Start date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"
    end_date_pattern = r"End date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"

    start_date_match = re.search(start_date_pattern, text)
    end_date_match = re.search(end_date_pattern, text)

    result = {}
    if start_date_match and end_date_match:
        result['Start date'] = start_date_match.group(1)
        result['End date'] = end_date_match.group(1)
    else:
        return None

    return result

Dates extraction with OLD METHOD

In [ ]:
insurances_folder_path = "/content/PDF-Metadata-Extractor/Insurances"

extracted_dates = {}

for root, dirs, files in os.walk(insurances_folder_path):
    for file_name in tqdm(files):
        full_file_path = os.path.join(root, file_name)

        extraction = PDFDatesFinderSemanticSearch(full_file_path)
        extraction.load_pdf()
        extraction.process_text()
        dates = extraction.find_dates()

        for value in dates.values():
          for phrase in value.values():
            if len(phrase):
              sentence = phrase[0]

              prompt = "Find start date and end date from the following sentence: " + sentence
              inputs = tokenizer(prompt, return_tensors="pt")
              generate_ids = model.generate(inputs.input_ids, max_length=1000)
              response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
              response = post_process_response_dates(response)

              if response:
                extracted_dates[str(file_name)] = response
                print(str(file_name) + ' EXTRACTED!')
              else:
                print(str(file_name) + ' NOT FOUND!')

Dates extraction with NEW METHOD

In [ ]:
insurances_folder_path = "/content/PDF-Metadata-Extractor/Insurances"

extracted_dates = {}

for root, dirs, files in os.walk(insurances_folder_path):
    for file_name in tqdm(files):
        full_file_path = os.path.join(root, file_name)

        extractor = PDFDatesFinderSpace(full_file_path)
        pages, tables = extractor.extract_mytext()
        paragraphs = [extractor.identify_paragraphs_space(page) for page in pages]
        #print(paragraphs)

        # keywords filter 1th level
        # Use a list comprehension with regex to keep only the phrases that contain the word "period" (case-insensitive)
        paragraphs = [sublist for sublist in paragraphs if any(re.search(r'\bperiod\b', phrase, re.IGNORECASE) for phrase in sublist)]

        # Now, further filter each sublist to keep only the phrases that contain the word "period"
        paragraphs = [[phrase for phrase in sublist if re.search(r'\bperiod\b', phrase, re.IGNORECASE)] for sublist in paragraphs]


        for phrase in paragraphs:
          if len(phrase):
            sentence = phrase[0]
            prompt = "Find start date and end date from the following sentence: " + sentence
            inputs = tokenizer(prompt, return_tensors="pt")
            generate_ids = model.generate(inputs.input_ids, max_length=200)
            response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            response = post_process_response_dates(response)

            if response:
                extracted_dates[str(file_name)] = response
                print(response)
                print(str(file_name) + ' EXTRACTED!')
            else:
              print(str(file_name) + ' NOT FOUND!')

  0%|          | 0/1 [00:00<?, ?it/s]

[['Period: 1 June 2022 to 1 December 2023'], ['PERIOD: From: 1 June 2022 To: 1 December 2023'], [], [], ['1 June 2022 and 1 December 2023'], [], [], [], [], [], [], [], [], [], [], [], ['ept in relation to the operation of any 50/50 Marine construction lo', '(b) at least 60 days (or such lesser period (if any) as may be specified from time to time by Insurers in the case of war risk and kindred perils) before any reduction in limits or coverage, any increase in deductibles or any termination before the original expiry date is to take effect; (c) at least 60 days before any suspension of any ins'], [], [], [], ['iod of the insurance policy is from 1.6.2022 to 30.11.2023 (inclusive) which shall a'], [], [], []]


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


{'Start date': '1 June 2022', 'End date': '1 December 2023'}
policy_9.pdf EXTRACTED!
{'Start date': '1 June 2022', 'End date': '1 December 2023'}
policy_9.pdf EXTRACTED!


  0%|          | 0/1 [02:03<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
extracted_dates

{'policy_8.pdf': {'Start date': '1st January 2021',
  'End date': '31st December 2021'},
 'policy_10.pdf': {'Start date': '1st January 2022',
  'End date': '31st December 2022'},
 'policy_2.pdf': {'Start date': '27 November 2020',
  'End date': '29 November 2020'},
 'policy_1.pdf': {'Start date': '31st March 2021',
  'End date': '30th March 2022'},
 'policy_9.pdf': {'Start date': '1st January 2020',
  'End date': '31st December 2020'},
 'policy_4.pdf': {'Start date': '1st May 2022', 'End date': '30th April 2023'},
 'policy_5.pdf': {'Start date': '01 November 2021',
  'End date': '31 October 2022'}}